In [39]:
INPUT_FILE = 'raw/2/train-v1.1.json_transformed.json'
DATABASE_FILE = 'app.db'

Read json

In [40]:
import json
from pathlib import Path

with open(Path(INPUT_FILE)) as file:
    txt = file.read()
    json_obj = json.loads(txt)

Write raw JSON to the database

In [41]:
from sqlite3 import connect

raw_doc_id = None
with connect(Path(DATABASE_FILE)) as db:
    cursor = db.cursor()
    cursor.execute('INSERT INTO main.import_raw(document) VALUES (?)', [txt])
    raw_doc_id = cursor.lastrowid
    db.commit()

In [42]:
raw_doc_id

1

In [43]:
len([el['ID'] for el in json_obj])

18896

Write JSON text to the db

In [44]:
texts_ids = {}
with connect(Path(DATABASE_FILE)) as db:
	cursor = db.cursor()
	for el in json_obj:
		values = (raw_doc_id, el['TEXT'])
		cursor.execute('INSERT INTO main.import_text(__import_raw__id, text) VALUES(?, ?)', values)
		text_id = cursor.lastrowid
		texts_ids[el['ID']] = text_id
	db.commit()

Write JSON questions to the db

In [47]:
with connect(Path(DATABASE_FILE)) as db:
	cursor = db.cursor()
	for el in json_obj:
		text_id = texts_ids[el['ID']]
		for q in el['QUESTIONS']:
			values = (text_id, q['QUESTION'], q['ANSWER'])
			cursor.execute('INSERT INTO main.import_questions(__import_text__id, question, answer) VALUES(?, ?, ?)', values)
	db.commit()